<a href="https://colab.research.google.com/github/Eugene2705/MITx_Supply_Chain_Optimisation/blob/main/SCM_275x_Transportation_Problem_Python_Exercise.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

SCM.275x - Advanced Supply Chain Systems Planning and Network Design
# **Transportation Problem - Python Exercise**

### *Before starting, make sure to save a copy of this notebook to your Google Drive!*

## **Initialization**

In [ ]:
# Install necessary packages if they are not already installed

!pip install gurobipy   # Gurobi optimization solver
!pip install pandas     # Pandas for data analysis and manipulation
!pip install folium     # Folium for creating interactive maps
!pip install geopy      # Geopy for computing distances and working with geographic data


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 32.0 MB/s eta 0:00:00


In [ ]:
# Import all required packages

import pandas as pd                   # For data manipulation and analysis
import gurobipy as grb                # Gurobi optimization library for solving mathematical models
import folium                         # For creating interactive maps
import folium.plugins as plugins      # Additional plugins for folium
from geopy.distance import geodesic   # For calculating geodesic distances between two points


## **Helper functions**

### **Ploting nodes on a map**

In [ ]:
# Defining a function to plot nodes on a map using folium

def plot_nodes(map,                         # Folium map object to plot the nodes on
               nodes,                       # Dictionary of node objects where each node contains attributes like latitude and longitude
               icon,                        # Icon symbol to use for the markers on the map
               color,                       # Color of the marker icon
               background_color,            # Background color of the marker icon
               ):

    # Loop through each node in the dictionary
    for node in nodes.values():

        # Create a folium marker
        marker = folium.Marker(
            location=[node.lat, node.lon],              # Set the marker's location
            popup = (node.ID + "-" + node.name),        # Create a marker popup with the node ID and name
            icon=plugins.BeautifyIcon(                  # Create a marker's icon
                icon=icon,
                icon_shape="circle",
                text_color=color,                       # Set the marker's color
                border_color=color,
                background_color=background_color,
            )
        )

        # Add a folium marker to the map
        marker.add_to(map)


### **Computing geodesic distance**

In [ ]:
# Defining a function for computing geodesic distances between two locations

def compute_geodesic_distance(origin,       # Origin node object
                              destination,  # Destination node
                              unit='km'):   # Unit ('km' or 'mi'), default value = 'km'

    # Extract coordinates (latitude and longitude) from origin and destination
    origin_coordinates = [origin.lat, origin.lon]
    destination_coordinates = [destination.lat, destination.lon]

    # Compute distance based on the specified unit ('km' or 'mi')
    if unit == 'km':
        distance = geodesic(origin_coordinates, destination_coordinates).km  # Compute distance in kilometers
    elif unit == 'mi':
        distance = geodesic(origin_coordinates, destination_coordinates).mi  # Compute distance in miles

    return distance  # Return the calculated distance


### **Ploting flows on a map**

In [ ]:
# Defining a function to plot flows on a map using folium

def plot_flows(map,                   # Folium map object where flows will be plotted.
               vars,                  # Dictionary of decision variables from the optimization model
               nodes,                 # Dictionary of node objects
               max_width = 30,        # Maximum line width for the flows, default is 30
               color = 'grey',        # Color of the lines representing flows, default is grey
               opacity = 0.5):        # Opacity of the lines, default is 0.5

    # Find the maximum flow value to normalize line widths
    max_val = max([var.X for (node1_key, node2_key), var in vars.items()])

    # Iterate over flow decision variables (keys represent node pairs)
    for (node1_key, node2_key), var in vars.items():

        # Plot only positive flows
        if var.X > 0:

            # Get the coordinates of the nodes for plotting the line
            points = [[nodes[node1_key].lat, nodes[node1_key].lon],
                      [nodes[node2_key].lat, nodes[node2_key].lon]]


            # Add a PolyLine to the map to represent the flow between the nodes
            folium.PolyLine(points,
                            color=color,                                # Set the color of the line
                            weight=var.X / max_val * max_width,         # Normalize line width based on flow value
                            opacity=opacity,                            # Set line opacity
                            popup=var.X).add_to(map)                    # Show the flow value in a popup on the map


## **Data setup and preprocessing**

### **Nodes**

#### Definition of Classes

In [ ]:
# Class representing a Customer object

class Customer():
    def __init__(self, ID, name, lat, lon, demand):
        self.ID = ID              # Customer's ID
        self.name = name          # Customer's name
        self.lat = lat            # Customer's latitude
        self.lon = lon            # Customer's longitude
        self.demand = demand      # Customer's demand


In [ ]:
# Class representing a Supplier object

class Supplier():
    def __init__(self, ID, name, lat, lon, supply):
        self.ID = ID            # Supplier's ID
        self.name = name        # Supplier's name
        self.lat = lat          # Supplier's latitude
        self.lon = lon          # Supplier's longitude
        self.supply = supply    # Supplier's available supply


#### Reading input files

In [ ]:
# File containing customer data
customer_data_file = 'https://raw.githubusercontent.com/scm275/problem_sets_scm275/main/transportation_problem/customers.csv'

# Loading customer data into a pandas DataFrame
customers_df = pd.read_csv(customer_data_file)

# Displaying the first few rows of the DataFrame to verify the data
customers_df.head()

,ID,name,lat,lon,demand
0,C01,Concord,43.206898,-71.537994,9000
1,C02,Trenton,40.220596,-74.769913,10000
2,C03,Austin,30.274670,-97.740349,10000
3,C04,Providence,41.830914,-71.414963,9000
4,C05,Montpelier,44.262436,-72.580536,7000


In [ ]:
# File containing supplier data
supplier_data_file = 'https://raw.githubusercontent.com/scm275/problem_sets_scm275/main/transportation_problem/suppliers.csv'

# Loading supplier data into a pandas DataFrame
suppliers_df = pd.read_csv(supplier_data_file)

# Displaying the first few rows of the DataFrame to verify the data
suppliers_df.head()


,ID,name,lat,lon,supply
0,S01,Columbia,34.000343,-81.033211,40000
1,S02,Jefferson City,38.579201,-92.172935,30000
2,S03,Santa Fe,35.682240,-105.939728,20000
3,S04,Richmond,37.538857,-77.433640,32000
4,S05,Carson City,39.163914,-119.766121,20000


#### Creating node objects

In [ ]:
# Initializing an empty dictionary to store node objects
nodes = dict()

In [ ]:
# Creating a dictionary of customer objects
customers = dict()
for i, row in customers_df.iterrows():
    customers[row['ID']] = Customer(ID=row['ID'],           # Customer's ID
                                    name=row['name'],       # Customer's name
                                    lat=row['lat'],         # Customer's latitude
                                    lon=row['lon'],         # Customer's longitude
                                    demand=row['demand'])   # Customer's demand

# Merging the customers dictionary into the existing nodes dictionary
nodes = {**nodes, **customers}

In [ ]:
# Creating a dictionary of supplier objects
suppliers = dict()
for i, row in suppliers_df.iterrows():
    suppliers[row['ID']] = Supplier(ID=row['ID'],           # Supplier's ID
                                    name=row['name'],       # Supplier's name
                                    lat=row['lat'],         # Supplier's latitude
                                    lon=row['lon'],         # Supplier's longitude
                                    supply=row['supply'])   # Supplier's available supply

# Merging the suppliers dictionary into the existing nodes dictionary
nodes = {**customers, **suppliers}

In [ ]:
# Verifying that the total supply is <= total demand

print('Total Supply:', sum([x.supply for x in suppliers.values()]))   # Calculate and print total supply
print('Total Demand:', sum([x.demand for x in customers.values()]))   # Calculate and print total demand

Total Supply: 171000
Total Demand: 171000


#### Visualizing node objects

In [ ]:
# Create a new map centered on the US with a zoom level of 5
map = folium.Map([40, -95.0], zoom_start=5)

# Plot customer locations with a store icon, green color, and yellow background
plot_nodes(map=map, nodes=customers, icon='store', color='green', background_color='yellow')

# Plot supplier locations with an industry icon, orange color, and yellow background
plot_nodes(map=map, nodes=suppliers, icon='industry', color='orange', background_color='yellow')

# Add a tile layer for better map visualization (cartodbpositron theme)
folium.TileLayer('cartodbpositron').add_to(map)

# Display the map with all the plotted data
map


### **Arcs**

#### Arc distances

In [ ]:
# Creating a dictionary containing distances between customers and suppliers
distances = dict()
for s, supplier in suppliers.items():                                                                   # Iterate over suppliers
    for c, customer in customers.items():                                                               # Iterate over customers
        distances[s, c] = compute_geodesic_distance(origin=customer, destination=supplier, unit='km')   # Calculate and store distance in kilometers


#### Arc costs

In [ ]:
cost_unit_km = 1.2  # Cost per unit per kilometer

# Creating a dictionary containing unit costs between customers and suppliers
unit_cost = dict()
for s, supplier in suppliers.items():                                                           # Iterate over suppliers
    for c, customer in customers.items():                                                       # Iterate over customers
        unit_cost[s, c] = distances[s, c] * cost_unit_km                                        # Calculate unit cost as distance multiplied by cost per km


## **Optimization model**

### **Creating and solving the optimization model**

***❗Task 1: modify the optimization model to account for distance-based cost***

Modify the optimization model below such that the cost of each arc is calculated as the product of the distance between the two nodes and the unit cost per distance (e.g., $ per unit-km).

***❗Task 2: modify the optimization model to account service level constraints***

Modify the optimization model below such that the maximum distance between suppliers and customers is 1500km.

In [ ]:
# Create a new Gurobi model
model = grb.Model("Transportation Problem")

# Creating decision variables

# Decision variable representing the flows between suppliers and customers
flow_vars = dict()  # Explicitly named to indicate they are decision variables
for s, supplier in suppliers.items():
    for c, customer in customers.items():
        flow_vars[s, c] = model.addVar(vtype=grb.GRB.CONTINUOUS,
                                       name="flow_vars_{0}_{1}".format(s, c))

# Creating the objective function expression
total_cost = grb.quicksum(unit_cost[s, c] * flow_vars[s, c]
                         for s, supplier in suppliers.items()
                         for c, customer in customers.items())

# Setting the objective function
model.setObjective(total_cost, grb.GRB.MINIMIZE)

# Adding demand constraints
for c, customer in customers.items():
    model.addConstr(grb.quicksum(flow_vars[s, c] for s, supplier in suppliers.items()) == customer.demand)

# Adding supply constraints
for s, supplier in suppliers.items():
    model.addConstr(grb.quicksum(flow_vars[s, c] for c, customer in customers.items()) <= supplier.supply)

# Solving the model
model.optimize()


Restricted license - for non-production use only - expires 2025-11-24
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (linux64 - "Ubuntu 22.04.3 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 26 rows, 120 columns and 240 nonzeros
Model fingerprint: 0x79bf3707
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+02, 5e+03]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e+03, 4e+04]
Presolve time: 0.02s
Presolved: 26 rows, 120 columns, 240 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.0615174e+08   7.100000e+04   0.000000e+00      0s
      14    1.4374377e+08   0.000000e+00   0.000000e+00      0s

Solved in 14 iterations and 0.03 seconds (0.00 work units)
Optimal objective  1.437437681e+08


## **Solution visualization and analysis**

In [ ]:
# Visualizing the customer and supplier locations on the map

# Create a new map centered on the US with a zoom level of 5
map = folium.Map([40, -95.0], zoom_start=5)

# Plot customer locations with a store icon, green color, and yellow background
plot_nodes(map=map, nodes=customers, icon='store', background_color='yellow', color='green')

# Plot supplier locations with an industry icon, orange color, and yellow background
plot_nodes(map=map, nodes=suppliers, icon='industry', background_color='yellow', color='orange')

# Plot the flows between suppliers and customers with a maximum line width of 20
plot_flows(map=map, max_width=20, vars=flow_vars, nodes=nodes)

# Add a tile layer for better map visualization (cartodbpositron theme)
folium.TileLayer('cartodbpositron').add_to(map)

# Display the map with all the plotted data
map
